In [1]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def delete_chroma_collection(collection_name='document_embeddings_v9', persist_directory='chroma_db_v4'):
    # Öffne die Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Überprüfe, ob die Collection existiert, und lösche sie, wenn ja
    existing_collections = [col.name for col in client.list_collections()]
    if collection_name in existing_collections:
        client.delete_collection(name=collection_name)
        print(f"Collection '{collection_name}' erfolgreich gelöscht.")
    else:
        print(f"Collection '{collection_name}' nicht gefunden.")
        
delete_chroma_collection(collection_name='document_embeddings_v9')

Collection 'document_embeddings_v9' erfolgreich gelöscht.


In [2]:
import pickle

def load_pickle_files():
    # Pfade zu den Pickle-Dateien
    chunked_doc_path = "data/pickle/chunked_doc_list.pkl"
    keywords_path = "data/pickle/chunk_keywords.pkl"
    
    # Laden der Pickle-Dateien
    with open(chunked_doc_path, 'rb') as doc_file:
        chunked_documents = pickle.load(doc_file)
    
    with open(keywords_path, 'rb') as keywords_file:
        keywords_per_chunk = pickle.load(keywords_file)
    
    return chunked_documents, keywords_per_chunk

def print_first_five_elements(list_obj):
    # Überprüfen, ob die Liste mindestens 5 Elemente hat
    for i, element in enumerate(list_obj[:5]):
        print(f"Element {i+1}: {element}")

# Benutzen der Funktionen
chunked_documents, keywords_per_chunk = load_pickle_files()

# Ausgabe der ersten 5 Elemente der beiden Listen
print("Erste 5 Elemente von chunked_documents:")
print_first_five_elements(chunked_documents)

print("\nErste 5 Elemente von keywords_per_chunk:")
print_first_five_elements(keywords_per_chunk)

Erste 5 Elemente von chunked_documents:
Element 1: Der blaue Altglascontainer das kommt hinein: • Saubere Flaschen und Deckelgläser nach den Farben Weiß, Grün und Braun getrennt; blaues oder buntes Glas bitte immer zum Grünglas geben. • Glasflaschen • Konservengläser das kommt nicht hinein: • Flachglas wie Spiegel und Scheiben (Restmüll oder Sperrmüll) • Trinkgläser und Blumenvasen (Restmüll)
Element 2: • Trinkgläser und Blumenvasen (Restmüll) • Porzellan und Keramik (Restmüll) • Glühbirnen (Restmüll), Energiesparlampen (Schadstoffmobil oder Wertstoffhöfe) • Stark verschmutzte Gläser (Restmüll) FES
Element 3: • Stark verschmutzte Gläser (Restmüll) FES Frankfurter Entsorgungsund Service GmbH Weidenbornstraße 40 60389 Frankfurt am Main Servicetelefon: 0800 20080070 Servicetelefax: 069 21231323 services@fesfrankfurt.de www.fesfrankfurt.de FESServicecenter Liebfrauenberg 52–54 (Ecke Töngesgasse/Schärfengässchen) 60313 Frankfurt am Main Mo. – Fr. 10.00 – 18.00 Uhr Sa. 10.00 – 16.00 Uhr was 

In [3]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
#from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

def create_embeddings(chunk):
    
    # Lade ein vortrainiertes Modell
    model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    # Erstelle Embeddings für die Texte
    chunk_embedding = model.encode(chunk)

    print("Embedding erstellt")
    return chunk_embedding

def create_chroma_v2(chunk, keywords, collection_name, persist_directory):
    # Erstelle oder öffne eine Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Erstelle eine neue Collection in der Datenbank oder lade eine bestehende
    collection = client.get_or_create_collection(name=collection_name)
    
    print(f"Chroma-Datenbank '{collection_name}' erfolgreich erstellt.")

    
    #chunk_embedding = create_embeddings(chunks)
    
    #collection.add(
    #embeddings = chunk_embedding.tolist(),  # Konvertiere Embeddings zu einer Liste von Listen
    #documents = [chunks],
    #metadatas = [{"keywords": "student info"}],
    #ids = ["id1"]
    #)
    
    # Angenommen, du hast bereits `chunked_documents` und `keywords_per_chunk` geladen
# und die Funktion `create_embeddings(chunks)` ist implementiert

    for i, (chunk, keywords) in enumerate(zip(chunked_documents, keywords_per_chunk)):
        # IDs werden dynamisch erzeugt, z.B. basierend auf dem Index
        doc_id = f"id_{i+1}"
    
        # Erzeuge Embeddings für den aktuellen Chunk
        chunk_embedding = create_embeddings(chunk)

        # Konvertiere die Embeddings in eine Liste von Listen
        embeddings = chunk_embedding.tolist()
    
        # Konvertiere die Liste von Keywords in einen String
        keywords_str = ", ".join(keywords)  # Keywords in einen einzigen String umwandeln
    
        # Hinzufügen des Chunks mit den entsprechenden Keywords und Embeddings
        collection.add(
            documents=[chunk],  # Der aktuelle Chunk
            metadatas=[{"keywords": keywords_str}],  # Keywords als String in den Metadaten
            ids=[doc_id],  # Eine eindeutige ID für jeden Eintrag
            embeddings=embeddings  # Die Embeddings als Liste von Listen
        )

    print("Alle Chunks wurden erfolgreich mit ihren Embeddings und Metadaten zu Chroma hinzugefügt.")
    
    
    # results = collection.query(
    # query_texts=["Was kommt in den gelbe Tonne?"],
    # n_results=2
    #)
    # print(results)
    # print(collection.get(include=['embeddings', 'documents', 'metadatas']))


    
collection_name='document_embeddings_v9'
persist_directory='chroma_db_v4'
test = """
Alexandra Thompson, a 19-year-old computer science sophomore with a 3.7 GPA,
is a member of the programming and chess clubs who enjoys pizza, swimming, and hiking
in her free time in hopes of working at a tech company after graduating from the University of Washington.
"""

create_chroma_v2(chunked_documents, keywords_per_chunk, collection_name, persist_directory)

/Volumes/data1/Programming/predictiv_chatbot/predictiv_chatbot/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Chroma-Datenbank 'document_embeddings_v9' erfolgreich erstellt.


/Volumes/data1/Programming/predictiv_chatbot/predictiv_chatbot/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Alle Chunks wurden erfolgreich mit ihren Embeddings und Metadaten zu Chroma hinzugefügt.


In [4]:
import chromadb
from sentence_transformers import SentenceTransformer

# Verbindungsparameter
persist_directory = 'chroma_db_v4'  # Pfad zum Speicherverzeichnis
DEFAULT_TENANT = "default_tenant"  # Name des Mandanten
DEFAULT_DATABASE = "default_database"  # Name der Datenbank

# Chroma Client erstellen
client = chromadb.PersistentClient(
    path=persist_directory,
    settings=chromadb.Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# Text-zu-Vektor Modell laden (z.B. SentenceTransformer)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')  # oder ein anderes Modell

# Text in Vektor umwandeln
def text_to_vector(text):
    return model.encode(text).tolist()  # Umwandlung in Liste

# Textabfrage durchführen
def query_chroma_with_text(client, collection_name, query_text, n_results=3):
    query_vector = text_to_vector(query_text)
    collection = client.get_collection(collection_name)
    # Abfrage mit Begrenzung auf die besten n Ergebnisse
    results = collection.query(query_embeddings=[query_vector], n_results=n_results)
    return results

# Beispielhafte Verwendung
query_text = "Was kommt in die gelbe Tonne?"  # Ihre Textabfrage
collection_name = 'document_embeddings_v9'  # Name der zu durchsuchenden Sammlung

# Abfrageergebnisse abrufen
results = query_chroma_with_text(client, collection_name, query_text)

# Nur die besten 3 Ergebnisse ausgeben (docs, keywords und score)
for i in range(len(results['documents'])):
    print(f"Ergebnis {i + 1}:")
    # Jedes Dokument, die zugehörigen Keywords und den Score anzeigen
    for doc, metadata, score in zip(results['documents'][i], results['metadatas'][i], results['distances'][i]):
        print("Dokument:", doc)
        print("Keywords:", metadata.get('keywords', 'Keine Keywords verfügbar'))
        print("Score:", score)
        print()  # Leerzeile zur Trennung der Ergebnisse

Ergebnis 1:
Dokument: • Stark verschmutzte FastFood-Verpackungen wie Pizzakartons (Restmüll) Hinweis: Ist zu viel Restmüll in der gelben Tonne, können die Wertstoffe nicht mehr aussortiert und weiterverarbeitet werden. Standort: Am Wohnhaus Leerung: 14täglich die grüne Altpapiertonne das kommt hinein: • In die grüne Altpapiertonne gehören Papier und Pappe – ohne Beschichtung und unverschmutzt. • Pappen und Kartonagen • Zeitungen, Illustrierte, Kataloge • Bücher und Hefte das kommt nicht hinein: • Stark verschmutztes Papier (Restmüll) •
Keywords: grüne, Altpapiertonne, Papier, FastFood-Verpackungen, Hinweis
Score: 16.188478962787908

Dokument: Verpackungen aus Metall wie Konservenund Getränkedosen • Geschäumte Kunststoffe, z. B. Obstund Gemüseverpackungen das kommt nicht hinein: • Verpackungen aus Papier und Pappe (Altpapiertonne) • Verpackungen aus Glas (Altglascontainer) • Stark verschmutzte FastFood-Verpackungen wie Pizzakartons (Restmüll) Hinweis: Ist zu viel Restmüll in der gelben 

In [ ]:
from sentence_transformers import SentenceTransformer

# Alternatives Modell, das öffentlich verfügbar ist
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Erstelle Embeddings für die Texte
chunks = ["Dein erster Textchunk", "Dein zweiter Textchunk"]  # Beispieltexte
students_embeddings = model.encode(chunks)

print(students_embeddings)